In [1]:
import nbimporter
from pathlib import Path
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split

from Preprop import * # load the data_preprocess function
from Feature_En import * # load the feature_generating function

In [2]:
d_path = 'datas'
params = {'depth': 10.0, 
        'l2_leaf_reg': 5.555897323622429, 
        'learning_rate': 0.4359843989797999, 
        'loss_function': 'Logloss', 
        'max_bin': 50.0, 
        'random_strength': 0.009790073389282446
}

In [3]:
def evaluate(model, X_test, y_test, model_name, use_proba=True):
    if use_proba == True:
        y_pred = model.predict_proba(X_test)[:, 1]
    elif use_proba == False:
        y_pred =  model.predict(X_test)
    ndcg = ndcg_score(y_test.values.reshape(1, -1), y_pred.reshape(1, -1), k=20)
    print(f'NDCG@{20} на тестовых данных c использованием {model_name}: {ndcg}')

In [4]:
df = data_preprocess(d_path, use_test=True)

D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x4 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [5]:
df = generate_features(df)

Feature_En.ipynb:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "            dataframes_profile.loc[:, item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')\n",
Feature_En.ipynb:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        for featur in category_vars:\n",
Feature_En.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

Выделяем индексы для тестовой и тренировочной выборки

In [6]:
df = df.loc[:, df.columns[3:]]
train_ind = df['target'][~pd.isnull(df['target'])].index
test_ind = df['target'][pd.isnull(df['target'])].index

Разбиение с помощью train_test_split на валидационную 33% и тренировочную выборку

In [7]:
X = df.drop('target', axis=1).loc[train_ind] # Х без тестовой
y = df['target'][train_ind] # Y без тестовой

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

# X_test = df.drop('target', axis=1).loc[test_ind].reset_index(drop=True)
# y_test = df['target'][test_ind]

In [8]:
# print(y_train)

# print(y_val)

Используя CatBoostClassifier обучаем модель и после получаем предикт с метрикой ndcg@20

In [9]:
model = CatBoostClassifier(n_estimators=50, random_state=0, **params)
model.fit(X_train, y_train)

evaluate(model, X_val, y_val, 'CatBoostClassifier', use_proba=False)
evaluate(model, X_val, y_val, 'CatBoostClassifier', use_proba=True)

0:	learn: 0.6443354	total: 2.18s	remaining: 1m 46s
1:	learn: 0.6254944	total: 4.12s	remaining: 1m 38s
2:	learn: 0.6175922	total: 6s	remaining: 1m 34s
3:	learn: 0.6133292	total: 7.94s	remaining: 1m 31s
4:	learn: 0.6104058	total: 10s	remaining: 1m 30s
5:	learn: 0.6074851	total: 12s	remaining: 1m 28s
6:	learn: 0.6052777	total: 13.9s	remaining: 1m 25s
7:	learn: 0.6034109	total: 16s	remaining: 1m 24s
8:	learn: 0.6021282	total: 18s	remaining: 1m 21s
9:	learn: 0.6009218	total: 19.9s	remaining: 1m 19s
10:	learn: 0.5992748	total: 22s	remaining: 1m 17s
11:	learn: 0.5980221	total: 24.1s	remaining: 1m 16s
12:	learn: 0.5970893	total: 26.1s	remaining: 1m 14s
13:	learn: 0.5961507	total: 28.1s	remaining: 1m 12s
14:	learn: 0.5948886	total: 30.3s	remaining: 1m 10s
15:	learn: 0.5939868	total: 32.4s	remaining: 1m 8s
16:	learn: 0.5929240	total: 34.5s	remaining: 1m 6s
17:	learn: 0.5918373	total: 36.8s	remaining: 1m 5s
18:	learn: 0.5909980	total: 39s	remaining: 1m 3s
19:	learn: 0.5898931	total: 41.2s	remaini

Конечная оценка при валидации в 33% от исходной выборки = 0.705501188394323